<a href="https://colab.research.google.com/github/JA-Bar/nlp-depression/blob/master/BERT_suicidal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive  

Mounted at /content/gdrive


In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -q tensorflow-text

     |████████████████████████████████| 4.3MB 32.5MB/s 


In [ ]:
!pip install -q tf-models-official
!pip install -q sklearn

     |████████████████████████████████| 1.6MB 26.4MB/s 
     |████████████████████████████████| 686kB 53.1MB/s 
     |████████████████████████████████| 1.2MB 49.8MB/s 
     |████████████████████████████████| 61kB 10.2MB/s 
     |████████████████████████████████| 38.2MB 77kB/s 
     |████████████████████████████████| 358kB 45.3MB/s 
     |████████████████████████████████| 102kB 14.5MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 645kB 44.9MB/s 
     |████████████████████████████████| 174kB 57.2MB/s 


In [ ]:
import os
import shutil
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization 
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow as tf
from sklearn.model_selection import train_test_split
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
np.set_printoptions(precision=3, suppress=True)
tf.get_logger().setLevel('ERROR')

In [ ]:
train=pd.read_csv ('/content/gdrive/MyDrive/Clasificador/out.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data2=train

In [ ]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]


In [ ]:
data2.head(10)

,text,classification,non-suicide,suicide,Unnamed: 4
0,"ex wife threaten suiciderecently leave wife good cheat twice lie much decide refuse go back . day ago , begin threaten suicide . tirelessly spend paat day talk keep hesitating want believe I come back . know lot people threaten order get way , but happen really ? suppose handle death hand ? still love wife but not deal get cheat constantly feeling insecure . I worried today may day hope much n...",1,0,1,NaN
1,weird not get affect compliment come someone know irl but feel really good internet stranger,0,1,0,NaN
2,finally almost ... never hear bad year ever . swear fuck god annoying,0,1,0,NaN
3,need helpjust help I cry hard,1,0,1,NaN
4,"I losthello , name adam I struggle year I afraid . past year thought suicide , fear , anxiety I close limit . I quiet long I scar come family feeling . year ago lose aunt trigger . everyday feeling hopeless , lose , guilty , remorseful thing I do life , but thought like little I experience life ? time I reveal feeling family break saw cut . watch get worried something portray average day make ...",1,0,1,NaN
5,"honetly idki not know I even . feel like nothing nowhere . feel either nothing unbearably sad . I ignore friend every opitunity . feel like I loose girlfriend . hurt everyone talk not anything good . I behind education , feel alone but first time not feeling I enjoy . no hope dream . care nothing , not family , not friend , not even girlfriend still love , complicate not word describe . would ...",1,0,1,NaN
6,"trigger warning excuse self inflict burn know crisis line use panic attack . know not healthy thing . but , . something stupid impulse . burn . really need help excuse father daughter know history together year . see bad but ! always cut ankle wrist , I think excuse would easy one cut . work car last night not self harm long time . without think , usual impulse lose moment . say touch somethin...",1,0,1,NaN
7,end tonight.i not anymore . quit .,1,0,1,NaN
8,"everyone want edgy make self conscious feel like not stand . draw yes play guitar but honestly feel like stick past , taste music rock alt metal 's 's not really make feel unique style but see friend classmates get into rap edm hard feel like fit . not feel like stand others copy style I would another quirky kid cringey phase . many friend say look good grunge style kind agree but hard continu...",0,1,0,NaN
9,"life year oldhello . year old balding male . hairline trash make matter bad head huge . bipolar , depression cripple social anxiety . balding cherry top . wear hat even room I alone not stop think . pop xanax day try numb pain work little bit but come crash back twice hard come . not know communicate people anymore not know keep relationship . use one popular kid but dad pass away feel into de...",1,0,1,NaN


In [ ]:
data_f = {'text': data2['text'], 'suicide':  data2['classification']}
df1 = pd.DataFrame(data_f)
#df1["suicide"] = np.nan  
#for i in range(0,len(df1['non-suicide'])):
#  if df1['non-suicide'][i]==0:
#     df1['suicide'][i]=int(1)
#  else:
#    df1['suicide'][i]=int(0)

In [ ]:
data_f = {'text': df1['text'], 'suicide':  df1['suicide']}
df = pd.DataFrame(data_f)
#df=data2

In [ ]:
df.head(10)

,text,suicide
0,"ex wife threaten suiciderecently leave wife good cheat twice lie much decide refuse go back . day ago , begin threaten suicide . tirelessly spend paat day talk keep hesitating want believe I come back . know lot people threaten order get way , but happen really ? suppose handle death hand ? still love wife but not deal get cheat constantly feeling insecure . I worried today may day hope much n...",1
1,weird not get affect compliment come someone know irl but feel really good internet stranger,0
2,finally almost ... never hear bad year ever . swear fuck god annoying,0
3,need helpjust help I cry hard,1
4,"I losthello , name adam I struggle year I afraid . past year thought suicide , fear , anxiety I close limit . I quiet long I scar come family feeling . year ago lose aunt trigger . everyday feeling hopeless , lose , guilty , remorseful thing I do life , but thought like little I experience life ? time I reveal feeling family break saw cut . watch get worried something portray average day make ...",1
5,"honetly idki not know I even . feel like nothing nowhere . feel either nothing unbearably sad . I ignore friend every opitunity . feel like I loose girlfriend . hurt everyone talk not anything good . I behind education , feel alone but first time not feeling I enjoy . no hope dream . care nothing , not family , not friend , not even girlfriend still love , complicate not word describe . would ...",1
6,"trigger warning excuse self inflict burn know crisis line use panic attack . know not healthy thing . but , . something stupid impulse . burn . really need help excuse father daughter know history together year . see bad but ! always cut ankle wrist , I think excuse would easy one cut . work car last night not self harm long time . without think , usual impulse lose moment . say touch somethin...",1
7,end tonight.i not anymore . quit .,1
8,"everyone want edgy make self conscious feel like not stand . draw yes play guitar but honestly feel like stick past , taste music rock alt metal 's 's not really make feel unique style but see friend classmates get into rap edm hard feel like fit . not feel like stand others copy style I would another quirky kid cringey phase . many friend say look good grunge style kind agree but hard continu...",0
9,"life year oldhello . year old balding male . hairline trash make matter bad head huge . bipolar , depression cripple social anxiety . balding cherry top . wear hat even room I alone not stop think . pop xanax day try numb pain work little bit but come crash back twice hard come . not know communicate people anymore not know keep relationship . use one popular kid but dad pass away feel into de...",1


In [ ]:
train=df
test=train.iloc[int(len(train)*.8):int(len(train))]
train=train.iloc[0:int(len(train)*.8)]
print(len(test),"  ",len(train),"  ",len(test)+len(train))

46415    185659    232074


In [ ]:
test=train.iloc[int(len(train)*.8):int(len(train))]
train=train.iloc[0:int(len(train)*.8)]
print(len(test),"  ",len(train),"  ",len(test)+len(train))

37132    148527    185659


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

train_text = train['text'].to_numpy()
train_seqs=train_text
train_labels = train['suicide'].to_numpy().flatten()
train_labels=train_labels.astype(int)
train_ds2 = tf.data.Dataset.from_tensor_slices((train_seqs,train_labels))
train_ds2 = train_ds2.shuffle(1).batch(batch_size)
train_ds2 = train_ds2.prefetch(buffer_size=0)
train_ds2 = tf.data.Dataset.from_tensor_slices((train_seqs,train_labels))
train_ds2 = train_ds2.shuffle(1).batch(batch_size)
train_ds2 = train_ds2.prefetch(buffer_size=AUTOTUNE)

ValueError: ignored

In [ ]:
test=train.iloc[int(len(train)*.9):int(len(train))]
train=train.iloc[0:int(len(train)*.8)]
print(len(test),"  ",len(train),"  ",len(test)+len(train))

In [ ]:
valid_text = test['text'].to_numpy()
valid_seqs = valid_text
valid_labels = test['suicide'].to_numpy().flatten()
valid_labels=valid_labels.astype(int)
val_ds2 = tf.data.Dataset.from_tensor_slices((valid_seqs,valid_labels))
val_ds2 = val_ds2.batch(batch_size)
val_ds2 = val_ds2.prefetch(buffer_size=AUTOTUNE)

In [ ]:
print(train_ds2)
print(val_ds2)

In [ ]:
for text_batch, label_batch in train_ds2.take(1):
  for i in range(3):
    print(text_batch)
    label = label_batch.numpy()[i]
    print(label_batch)

In [ ]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [ ]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()

In [ ]:
tf.keras.utils.plot_model(classifier_model)

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
epochs = 4
steps_per_epoch = tf.data.experimental.cardinality(train_ds2).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds2,
                               validation_data=val_ds2,
                               epochs=epochs)

In [ ]:
loss, accuracy = classifier_model.evaluate(val_ds2)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# "bo" is for "blue dot"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
saved_model_path = '/content/gdrive/MyDrive/Clasificador/bert_suicidal'
classifier_model.save(saved_model_path, include_optimizer=False)

In [ ]:
saved_model_path = '/content/gdrive/MyDrive/Clasificador/bert_suicidalx.h5'
classifier_model.save(saved_model_path, include_optimizer=False)

In [ ]:
reloaded_model = tf.saved_model.load(saved_model_path)

In [ ]:
def bert_prediction(inputs, path):
  classification=[]
  reloaded_model = tf.saved_model.load(path)
  results=tf.sigmoid(reloaded_model(tf.constant(inputs)))
  print(results)
  #results1=tf.keras.activations.hard_sigmoid(reloaded_model(tf.constant(inputs)))
  #results2=tf.keras.activations.relu(reloaded_model(tf.constant(inputs)))
  #results3=tf.keras.activations.softsign(reloaded_model(tf.constant(inputs)))
  #results4=tf.keras.activations.swish(reloaded_model(tf.constant(inputs)))
  for i in range(len(inputs)):
    if float(results[i][0])<.50:
      classification.append(0)
    else:
      classification.append(1)
  return classification

In [ ]:
bert_prediction(['I bought a new computer','I want to died','Horrible day just let me go'], saved_model_path)